In [1]:
#导入一些必要的包
import os
import random
import paddle 
from paddle import nn
from paddle import optimizer as optim 
from paddle import fluid

#import paddle.vision.datasets as dset
#import paddle.vision.transforms as transforms
import numpy as np
from matplotlib import pyplot as plt 
from matplotlib import animation 
#import matplotlib.animation as animation

from tqdm import tqdm
from util.model import Generator,Discriminator,GeluGenerator,Generator256, SELUGenerator

#from util.config import BATCH_SIZE, INPUT_SIZE, MAX_VAL,STEPS,ADV_MUL,OUTPUT_SIZE


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/__init__.py:107: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import MutableMapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/rcsetup.py:20: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable, Mapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/colors.py:53: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sized


In [2]:
INPUT_SIZE = 32
OUTPUT_SIZE = 256
OUTPUT_SIZE_512 = 512

#OUTPUT_BITS = 16
MAX_VAL = 256
ADV_MUL = 3
BATCH_SIZE = 1024
LEARNING_RATE = 0.02
#DISC_WIDTH = 32

STEPS = 3000
# main settings
NUMBER_OF_BITS_PRODUCED = 1024000

In [3]:
#首先需要安装最新版本的visualdl
!pip install visualdl

Looking in indexes: https://mirror.baidu.com/pypi/simple/, https://mirrors.aliyun.com/pypi/simple/, https://pypi.tuna.tsinghua.edu.cn/simple/

[notice] A new release of pip available: 22.1.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [4]:
def reference_distribution_np(batch_size, seq_length, max_val) -> np.ndarray:
    """ Returns a batch of inputs sampled from the reference distribution, i.e.
    from the random uniform distribution. """
    return np.random.uniform(size=[batch_size, seq_length], low=0, high=max_val)

#产生从0到max_val的随机数，类型为paddle.Tensor,大小为[batch_size, seq_length]，其中floor为向下取整
def reference_distribution_paddle(batch_size, seq_length, max_val) -> paddle.Tensor:
    #return fluid.layers.uniform_random(shape=[batch_size, seq_length], min=0, max=max_val).floor()
    return fluid.layers.uniform_random(shape=[batch_size, seq_length], min=0, max=max_val)

In [5]:
def gradient_penalty(discriminator, real, fake, batchsize):
    t = paddle.uniform((batchsize,1,1))
    t = paddle.expand_as(t, real)
    inter = t * real +  (1-t) * fake
    inter.stop_gradient = False
    inter_ = discriminator(inter)
    grads = paddle.grad(inter_, [inter])[0]
    grads = paddle.reshape(grads, [batchsize, grads.shape[1], grads.shape[2]])
    epsilon = 1e-12
    norm = paddle.sqrt(
        paddle.mean(paddle.square(grads), axis=1) + epsilon
    )
    gp = paddle.mean((norm - 1)**2) * 10

    return gp

In [6]:
#参数初始化的模块
@paddle.no_grad()
def normal_(x, mean=0., std=1.):
    temp_value = paddle.normal(mean, std, shape=x.shape)
    x.set_value(temp_value)
    return x

@paddle.no_grad()
def uniform_(x, a=-1., b=1.):
    temp_value = paddle.uniform(min=a, max=b, shape=x.shape)
    x.set_value(temp_value)
    return x

@paddle.no_grad()
def constant_(x, value):
    temp_value = paddle.full(x.shape, value, x.dtype)
    x.set_value(temp_value)
    return x

def weights_init1(m):
    classname = m.__class__.__name__
    print(classname)
    if hasattr(m, 'weight') and classname.find('Conv') != -1:
        normal_(m.weight, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        normal_(m.weight, 0.,1.2)
        constant_(m.bias, 0)
    elif classname.find('Linear') != -1:
        uniform_(m.weight, -2., 2.)
        #constant_(m.bias, 0)
        uniform_(m.bias, -1.,1.)


def weights_init2(m):
    classname = m.__class__.__name__
    print(classname)
    if hasattr(m, 'weight') and classname.find('Conv') != -1:
        normal_(m.weight, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        normal_(m.weight, 1.0, 0.02)
        constant_(m.bias, 0)
    elif classname.find('Linear') != -1:
        uniform_(m.weight, -1, 1.)
        constant_(m.bias, 0)

def weights_init(m):
    classname = m.__class__.__name__
    print(classname)
    if hasattr(m, 'weight') and classname.find('Conv') != -1:
        normal_(m.weight, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        normal_(m.weight, 1.0, 0.02)
        constant_(m.bias, 0)
    elif classname.find('Linear') != -1:
        uniform_(m.weight, -1, 1.)
        constant_(m.bias, 0)

In [7]:
from util import mylayer
# Generator Code
'''
# Generator Code
class BigGeluGenerator(nn.Layer):
    def __init__(self, ):
        super(BigGeluGenerator, self).__init__()


        self.gen = nn.Sequential(

            #第一个input is Z,
            nn.Linear(INPUT_SIZE,256),
            #nn.BatchNorm(256),
            nn.GELU(),
            #nn.LeakyReLU(),


            #第二个
            nn.Linear(256,512),
            #nn.BatchNorm(500),
            nn.GELU(),
            #nn.LeakyReLU(),


            #第三个
            nn.Linear(512,1024),
            #nn.BatchNorm(1000),
            nn.GELU(),
            #nn.LeakyReLU(),


            #第三个
            nn.Linear(1024,2048),
            #nn.BatchNorm(2000),
            nn.GELU(),
            #nn.LeakyReLU(),


            nn.Linear(2048,4096),
            #nn.BatchNorm(4000),
            nn.GELU(),
            #nn.LeakyReLU(),


            #第三个
            nn.Linear(4096,2048),
            #nn.BatchNorm(2500),
            nn.GELU(),
            #nn.LeakyReLU(),

            #第三个
            nn.Linear(2048,512),
            #nn.BatchNorm(800),
            nn.GELU(),
            #nn.LeakyReLU(),

            #第四个
            nn.Linear(512,OUTPUT_SIZE),
            #nn.BatchNorm(OUTPUT_SIZE),
            mylayer.Mod(),
            #nn.BatchNorm(OUTPUT_SIZE),
            #mylayer.SoftSign128(),
            #mylayer.Tanh128(),
            #mylayer.Sigmoid256(),
            #nn.LayerNorm(OUTPUT_SIZE),

        )

    def forward(self, x):

        print(x)
        
        x = self.gen(x)

        print(x)

        return x

netG = BigGeluGenerator()

'''
# Generator Code
class GeluGenerator(nn.Layer):
    def __init__(self, ):
        super(GeluGenerator, self).__init__()

        self.gen = nn.Sequential(

            #第一个input is Z,
            nn.Linear(INPUT_SIZE,128),
            #nn.BatchNorm(100),
            nn.ReLU(),

            #第二个
            nn.Linear(128,256),
            #nn.BatchNorm(200),
            nn.ReLU(),


            #第三个
            nn.Linear(256,512),
            #nn.BatchNorm(400),
            nn.ReLU(),

            #第三个
            nn.Linear(512,1024),
            #nn.BatchNorm(400),
            nn.ReLU(),




            #第三个
            nn.Linear(1024,512),
            #nn.BatchNorm(400),
            nn.ReLU(),

            #第四个
            nn.Linear(512,OUTPUT_SIZE),
            #nn.BatchNorm(OUTPUT_SIZE),
            mylayer.Mod(),

        )

    def forward(self, x):

        x = self.gen(x)

        return x


#netG = SELUGenerator()
#netG = Generator256()
netG = GeluGenerator()
# Apply the weights_init function to randomly initialize all weights
#  to mean=0, stdev=0.2.
netG.apply(weights_init)

# Print the model
print(netG)

W1204 10:33:11.466737  8670 gpu_resources.cc:61] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 12.0, Runtime API Version: 11.2
W1204 10:33:11.471947  8670 gpu_resources.cc:91] device: 0, cuDNN Version: 8.2.


Linear
ReLU
Linear
ReLU
Linear
ReLU
Linear
ReLU
Linear
ReLU
Linear
Mod
Sequential
GeluGenerator
GeluGenerator(
  (gen): Sequential(
    (0): Linear(in_features=32, out_features=128, dtype=float32)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=256, dtype=float32)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=512, dtype=float32)
    (5): ReLU()
    (6): Linear(in_features=512, out_features=1024, dtype=float32)
    (7): ReLU()
    (8): Linear(in_features=1024, out_features=512, dtype=float32)
    (9): ReLU()
    (10): Linear(in_features=512, out_features=256, dtype=float32)
    (11): Mod(max_val=256.0,name=)
  )
)


In [8]:
seed_noise = reference_distribution_paddle(BATCH_SIZE, INPUT_SIZE, MAX_VAL)

print(seed_noise)
            #fake_sample = netG(seed_noise).floor()
fake_sample = netG(seed_noise)
print(fake_sample)

arr = fake_sample.numpy()
mean = np.mean(arr)
print("均值：", mean)

# 计算方差
variance = np.var(arr)
print("方差：", variance)

Tensor(shape=[1024, 32], dtype=float32, place=Place(gpu:0), stop_gradient=True,
       [[26.16350937 , 131.25456238, 37.71149445 , ..., 148.56259155,
         2.84108591  , 239.83857727],
        [61.13263321 , 228.86427307, 76.50019836 , ..., 84.59951019 ,
         21.86028671 , 10.44218159 ],
        [56.79540253 , 123.99736023, 217.20736694, ..., 219.81364441,
         51.49184418 , 70.88102722 ],
        ...,
        [198.68849182, 100.81215668, 34.93161011 , ..., 13.34265518 ,
         8.37232399  , 125.64051056],
        [107.50849152, 88.80255890 , 53.48756027 , ..., 102.76641846,
         120.09224701, 142.50074768],
        [16.27554893 , 216.37310791, 32.55152893 , ..., 148.78871155,
         185.09587097, 16.28174973 ]])
Tensor(shape=[1024, 256], dtype=float32, place=Place(gpu:0), stop_gradient=True,
       [[148., 58. , 176., ..., 4.  , 44. , 96. ],
        [56. , 199., 220., ..., 244., 12. , 144.],
        [154., 148., 176., ..., 216., 68. , 4.  ],
        ...,
        [12

In [9]:
# Generator Code
'''
class Discriminator(nn.Layer):
    def __init__(self, ):
        super(Discriminator, self).__init__()


        self.gen = nn.Sequential(

            #第一个input is Z,
            nn.Linear(OUTPUT_SIZE,256),
            #nn.BatchNorm(100),
            nn.ReLU(),

            #第二个
            nn.Linear(256,512),
            #nn.BatchNorm(200),
            nn.ReLU(),


            #第三个
            nn.Linear(512,1024),
            #nn.BatchNorm(400),
            nn.ReLU(),

            #第三个
            nn.Linear(1024,512),
            #nn.BatchNorm(400),
            nn.ReLU(),

            #第四个
            nn.Linear(512,1),
            #nn.BatchNorm(1),
            nn.Sigmoid(),

        )

    def forward(self, x):
        
        x = self.gen(x)

        return x

'''
class Discriminator(nn.Layer):
    def __init__(self,):
        super(Discriminator, self).__init__()
        self.dis = nn.Sequential(

            #第一个conv1d
            nn.Conv1D(1,4,2),
            nn.LeakyReLU(),
            #第二个conv1d
            nn.Conv1D(4,4,2),
            nn.LeakyReLU(),
            #第三个conv1d
            nn.Conv1D(4,4,2),
            nn.LeakyReLU(),
            #第四个conv1d
            nn.Conv1D(4,4,2),
            nn.LeakyReLU(),

            #池化层
            nn.MaxPool1D(2,1),

            #拉平
            nn.Flatten(),

            #第1个全连接层
            nn.Linear(1004,4),
            #nn.LeakyReLU(),

            #第2个全连接层
            nn.Linear(4,1),
            nn.Sigmoid(),
            #nn.LeakyReLU()
        )

    def forward(self, x):
        return self.dis(x)

netD = Discriminator()
netD.apply(weights_init)
print(netD)

Conv1D
LeakyReLU
Conv1D
LeakyReLU
Conv1D
LeakyReLU
Conv1D
LeakyReLU
MaxPool1D
Flatten
Linear
Linear
Sigmoid
Sequential
Discriminator
Discriminator(
  (dis): Sequential(
    (0): Conv1D(1, 4, kernel_size=[2], data_format=NCL)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Conv1D(4, 4, kernel_size=[2], data_format=NCL)
    (3): LeakyReLU(negative_slope=0.01)
    (4): Conv1D(4, 4, kernel_size=[2], data_format=NCL)
    (5): LeakyReLU(negative_slope=0.01)
    (6): Conv1D(4, 4, kernel_size=[2], data_format=NCL)
    (7): LeakyReLU(negative_slope=0.01)
    (8): MaxPool1D(kernel_size=2, stride=1, padding=0)
    (9): Flatten()
    (10): Linear(in_features=1004, out_features=4, dtype=float32)
    (11): Linear(in_features=4, out_features=1, dtype=float32)
    (12): Sigmoid()
  )
)


In [10]:
#Initialize BCELoss function
# 这里为需要改变的地方
loss = nn.BCELoss()   # DCGAN
#loss=nn.MSELoss()       # LSGAN
#  the progression of the generator
seed_noise = reference_distribution_paddle(BATCH_SIZE, INPUT_SIZE, MAX_VAL) #用来产生随机数的seed，一开始的短随机数
#print(seed_noise)

# Establish convention for real and fake labels during training
real_label = 1.
fake_label = 0.

# Setup Adam optimizers for both G and D
optimizerD = optim.Adam(parameters=netD.parameters(), learning_rate=0.0001, beta1=0.9999, beta2=0.9999)
optimizerG = optim.Adam(parameters=netG.parameters(), learning_rate=0.0001, beta1=0.9999, beta2=0.9999)
#optimizerD = optim.Adam(parameters=netD.parameters(), learning_rate=0.001, beta1=0.9999, beta2=0.9999)
#optimizerG = optim.Adam(parameters=netG.parameters(), learning_rate=0.001, beta1=0.9999, beta2=0.9999)

In [11]:
from visualdl import LogWriter
Dwriter = LogWriter(logdir='/home/aistudio/visualdl/dvisual')  #判别器可视化路径
Gwriter = LogWriter(logdir='/home/aistudio/visualdl/gvisual')  #生成器可视化路径

In [12]:
#训练
'''
ADV_MUL = 1
def train(outparmfile,STEPS):
    losses = [[], []]
    #plt.ion()
    now = 0
    for pass_id in range(STEPS):
        ############################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################

        #errD = 0.
        for i in range(ADV_MUL):
            
            optimizerD.clear_grad()

            real_sample = reference_distribution_paddle(BATCH_SIZE, OUTPUT_SIZE, MAX_VAL)
            #if pass_id %100 ==0:
                #print("real_sample",real_sample.numpy()[0])
            label = paddle.full((BATCH_SIZE, 1), real_label, dtype='float32')
            real_sample = paddle.reshape(real_sample, [BATCH_SIZE,1,OUTPUT_SIZE])
            real_out = netD(real_sample)

            real_loss =  -1 * paddle.mean(real_out)
            #errD_real.backward()


            seed_noise = reference_distribution_paddle(BATCH_SIZE, INPUT_SIZE, MAX_VAL)
            #fake_sample = netG(seed_noise).floor()
            fake_sample = netG(seed_noise)


            if pass_id %100 ==0:
                print("fake",fake_sample.numpy()[0].astype(int))
            #label = paddle.full((BATCH_SIZE, 1), fake_label, dtype='float32')


            fake_sample = paddle.reshape(fake_sample, [BATCH_SIZE,1,OUTPUT_SIZE])
            fake_out = netD(fake_sample.detach())
            fake_loss = paddle.mean(fake_out)

            gp = gradient_penalty(netD, real_sample, fake_sample, real_sample.shape[0])
            d_loss = fake_loss + real_loss + gp 

            #print("fake_loss",fake_loss.numpy(),"real_loss",real_loss.numpy(),"gp",gp.numpy())


            d_loss.backward()
            optimizerD.step()
            optimizerD.clear_grad()

            if i== ADV_MUL-1:
            #    d_loss = fake_loss + real_loss
                if pass_id %100 ==0:
                    print(fake_loss.numpy()[0],real_loss.numpy()[0])

        #print(errD.numpy()[0])    
        losses[0].append(d_loss.numpy()[0])

        ############################
        # (2) Update G network: maximize log(D(G(z)))
        ###########################

        optimizerG.clear_grad()
        seed_noise = reference_distribution_paddle(BATCH_SIZE, INPUT_SIZE, MAX_VAL)
        #fake = netG(seed_noise).floor()
        fake = netG(seed_noise)

        #label = paddle.full((BATCH_SIZE, 1), real_label, dtype='float32')
        fake = paddle.reshape(fake, [BATCH_SIZE,1,OUTPUT_SIZE])
        output = netD(fake)

        g_loss = -1*paddle.mean(output)
        g_loss.backward()
        optimizerG.step()
        optimizerG.clear_grad()

        losses[1].append(g_loss.numpy()[0])




        ############################
        # visualize
        ###########################

        now += 1

        #日志中增加损失
        Dwriter.add_scalar(tag="train/loss", step=now, value=d_loss.numpy()[0])
        Gwriter.add_scalar(tag="train/loss", step=now, value=g_loss.numpy()[0])

        
        if pass_id %100 ==0:
            msg = 'Epoch ID={0} \n\n D-Loss={1} G-Loss={2}'.format(pass_id, d_loss.numpy()[0], g_loss.numpy()[0])
            print(msg)


    paddle.save(netG.state_dict(),outparmfile)

    file_paths = ['lossfile1.txt', 'lossfile2.txt']
    for i in range(len(loss)):
        data = loss[i]
        file_path = file_paths[i]

        # 将数据按行组合成字符串
        content = '\n'.join(data)

        # 将数据写入txt文件
        with open(file_path, 'w') as file:
            file.write(content)

        print(f"Data has been written to the file: {file_path}")
'''

'\nADV_MUL = 1\ndef train(outparmfile,STEPS):\n    losses = [[], []]\n    #plt.ion()\n    now = 0\n    for pass_id in range(STEPS):\n        ############################\n        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))\n        ###########################\n\n        #errD = 0.\n        for i in range(ADV_MUL):\n            \n            optimizerD.clear_grad()\n\n            real_sample = reference_distribution_paddle(BATCH_SIZE, OUTPUT_SIZE, MAX_VAL)\n            #if pass_id %100 ==0:\n                #print("real_sample",real_sample.numpy()[0])\n            label = paddle.full((BATCH_SIZE, 1), real_label, dtype=\'float32\')\n            real_sample = paddle.reshape(real_sample, [BATCH_SIZE,1,OUTPUT_SIZE])\n            real_out = netD(real_sample)\n\n            real_loss =  -1 * paddle.mean(real_out)\n            #errD_real.backward()\n\n\n            seed_noise = reference_distribution_paddle(BATCH_SIZE, INPUT_SIZE, MAX_VAL)\n            #fake_sample = netG(see

In [13]:

ADV_MUL = 1
GEN_MUL = 1

#训练
def train(outparmfile,STEPS):
    losses = [[], []]
    #plt.ion()
    now = 0
    for pass_id in range(STEPS):
        ############################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################

        #errD = 0.
        for i in range(ADV_MUL):
            
            optimizerD.clear_grad()

            real_sample = reference_distribution_paddle(BATCH_SIZE, OUTPUT_SIZE, MAX_VAL)
            #if pass_id %100 ==0:
                #print("real_sample",real_sample.numpy()[0])
            label = paddle.full((BATCH_SIZE, 1), real_label, dtype='float32')
            real_sample = paddle.reshape(real_sample, [BATCH_SIZE,1,OUTPUT_SIZE])
            real_out = netD(real_sample)
            errD_real = loss(real_out, label)
            errD_real.backward()


            seed_noise = reference_distribution_paddle(BATCH_SIZE, INPUT_SIZE, MAX_VAL)
            #fake_sample = netG(seed_noise).floor()
            fake_sample = netG(seed_noise)
            if pass_id %100 ==0:
                print(fake_sample.numpy()[0].astype(int))
            label = paddle.full((BATCH_SIZE, 1), fake_label, dtype='float32')
            fake_sample = paddle.reshape(fake_sample, [BATCH_SIZE,1,OUTPUT_SIZE])
            fake_out = netD(fake_sample.detach())
            errD_fake = loss(fake_out,label)
            errD_fake.backward()
            optimizerD.step()
            optimizerD.clear_grad()

            if i== ADV_MUL-1:
                errD = errD_real + errD_fake
                #if pass_id %100 ==0:
                    #print(errD_real.numpy()[0],errD_fake.numpy()[0])

        #print(errD.numpy()[0])    
        losses[0].append(errD.numpy()[0])

        ############################
        # (2) Update G network: maximize log(D(G(z)))
        ###########################

        optimizerG.clear_grad()
        seed_noise = reference_distribution_paddle(BATCH_SIZE, INPUT_SIZE, MAX_VAL)
        #fake = netG(seed_noise).floor()
        fake = netG(seed_noise)
        label = paddle.full((BATCH_SIZE, 1), real_label, dtype='float32')
        fake = paddle.reshape(fake, [BATCH_SIZE,1,OUTPUT_SIZE])
        output = netD(fake)
        errG = loss(output,label)
        errG.backward()
        optimizerG.step()
        optimizerG.clear_grad()

        losses[1].append(errG.numpy()[0])




        ############################
        # visualize
        ###########################

        now += 1

        #日志中增加损失
        Dwriter.add_scalar(tag="train/loss", step=now, value=errD.numpy()[0])
        Gwriter.add_scalar(tag="train/loss", step=now, value=errG.numpy()[0])

        
        if pass_id %100 ==0:
            msg = 'Epoch ID={0} \n\n D-Loss={1} G-Loss={2}'.format(pass_id, errD.numpy()[0], errG.numpy()[0])
            print(msg)


    paddle.save(netG.state_dict(),outparmfile)


In [14]:
#STEPS = [3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000]
STEPS = [50000]
for step in STEPS:
    for i in range(1):
        outparmfile ="work/"+'SoftSignBignewnewbce'+str(i)+"generator-steps"+str(step)+".params"
        train(outparmfile, step)

 140 137 151 251  12 136 254  71  26  56 150 240  92 208 108 119  50 228
 248 143 116 206 185 154 137 244 113   3 186 142   5   0 218 209 218  62
 224 248  41 249  50 161 206 206 228  24 129 143  41 186 147 136  35  38
  40  77   4 148 250 229 188  13  30 124  26  85  64   6  21  98  21 104
  10 102  76 196 188 196  84 105 190   7 224 214 214  70  60  76 106 114
 200  84 254 113]
Epoch ID=44400 

 D-Loss=0.4919443428516388 G-Loss=7.24340295791626
[128 126 170 174 146  17  38  52 154 174 253 148  26 136  48 169 212 251
 106 214 157 168 200 160 236  71 149 111 253  94  12 176  20 246   5  13
 227  30  33   6 106  12  78  98  76  98 152 106 141 222 234 226 170 128
 234 146  85 190  44  12  48 212 145  80  18 116 126 151 169 136   6  96
 163 208  66  40  98 134 239 216 128  86 243  21 129  98  63  14 238 102
 189   4 219  84 106 114   4 157  13  19  44 186  67  76 240 187 176  10
  73 188 205 236  30 249 248 174  96 206   4 160 224 110  65  20  96 152
 141 167  16 121  77 188  57  38 242 2